Copyright

Jelen forráskód a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott "Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült.

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot GPLv3 licensz védi. Újrafelhasználás esetén lehetőség szerint kérjük az alábbi szerzőt értesíteni.

2021 (c) Gyires-Tóth Bálint, Zainkó Csaba (zainko kukac tmit pont bme pont hu)

Felhasználva: https://keras.io/examples/nlp/lstm_seq2seq/

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
#Magyar nyelvű párhuzamos korpusz letöltése
!curl -O http://www.manythings.org/anki/hun-eng.zip
!unzip hun-eng.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3482k  100 3482k    0     0  2292k      0  0:00:01  0:00:01 --:--:-- 2292k
Archive:  hun-eng.zip
  inflating: _about.txt              
  inflating: hun.txt                 


In [3]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 25000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "hun.txt"


In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    #Eltároljuk, hogy a mondatpárokban milyen karakterek fordulnak elő
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
#A tanító adatokat 3 tömbben tároljuk el one-hot kódolással:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


Number of samples: 25000
Number of unique input tokens: 73
Number of unique output tokens: 92
Max sequence length for inputs: 19
Max sequence length for outputs: 56


In [5]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [6]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
#Tanítás során a validation accuracy-t maximalizáljuk
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
tb = TensorBoard(log_dir='logs', histogram_freq=1, write_graph=1)
es = EarlyStopping(monitor="val_accuracy",mode='max',patience=20, verbose=1)
cp = ModelCheckpoint(filepath='s2s', save_best_only=True, verbose=1,monitor='val_accuracy',mode='max')
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[es,cp,tb]
)



Epoch 1/100
313/313 [==============================] - ETA: 0s - loss: 1.0323 - accuracy: 0.7433
Epoch 00001: val_accuracy improved from -inf to 0.74583, saving model to s2s


INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


313/313 [==============================] - 31s 70ms/step - loss: 1.0323 - accuracy: 0.7433 - val_loss: 0.9154 - val_accuracy: 0.7458
Epoch 2/100
313/313 [==============================] - ETA: 0s - loss: 0.6954 - accuracy: 0.8015
Epoch 00002: val_accuracy improved from 0.74583 to 0.77919, saving model to s2s


INFO:tensorflow:Assets written to: s2s/assets


...

INFO:tensorflow:Assets written to: s2s/assets


313/313 [==============================] - 21s 67ms/step - loss: 0.2571 - accuracy: 0.9255 - val_loss: 0.5085 - val_accuracy: 0.8617
Epoch 24/100
313/313 [==============================] - ETA: 0s - loss: 0.2500 - accuracy: 0.9275
Epoch 00024: val_accuracy improved from 0.86174 to 0.86246, saving model to s2s


INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


313/313 [==============================] - 21s 66ms/step - loss: 0.2500 - accuracy: 0.9275 - val_loss: 0.5145 - val_accuracy: 0.8625
Epoch 25/100
313/313 [==============================] - ETA: 0s - loss: 0.2427 - accuracy: 0.9297
Epoch 00025: val_accuracy did not improve from 0.86246
313/313 [==============================] - 11s 36ms/step - loss: 0.2427 - accuracy: 0.9297 - val_loss: 0.5220 - val_accuracy: 0.8613
Epoch 26/100
313/313 [==============================] - ETA: 0s - loss: 0.2363 - accuracy: 0.9315
Epoch 00026: val_accuracy did not improve from 0.86246
313/313 [==============================] - 11s 34ms/step - loss: 0.2363 - accuracy: 0.9315 - val_loss: 0.5253 - val_accuracy: 0.8604
Epoch 27/100
313/313 [==============================] - ETA: 0s - loss: 0.2300 - accuracy: 0.9332
Epoch 00027: val_accuracy did not improve from 0.86246
313/313 [==============================] - 11s 36ms/step - loss: 0.2300 - accuracy: 0.9332 - val_loss: 0.5270 - val_accuracy: 0.8606
Epoch 28/

In [7]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

#Encoder
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

#Decoder
decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


In [8]:
#Fordítás:
input_text="I love dogs."
input_seq= np.zeros((1,max_encoder_seq_length, num_encoder_tokens), dtype="float32")
for i,ch in enumerate(input_text):
  input_seq[0,i,input_token_index[ch]]=1.0
#Szóköz a padding karakter:
input_seq[0,i+1:,input_token_index[" "]]=1.0
decoded_sentence = decode_sequence(input_seq)

print("Input sentence:", input_text)
print("Decoded sentence:", decoded_sentence)

Input sentence: I love dogs.
Decoded sentence: Szeretem a kutyákat.

